# Final Project: Data Preperation, Cleaning, Extraction, and Staging.

**Buckets + Data Prep:** for this text classification, we will be doing a sentiment analysis of chuncks of documents. To prepare the data, we'll import the markdown files and split them by section. titles of sections will be included in the section following the title. Each section will be tokenized and we will try to keep batches reletivly the same size. The buckets will be: active, proactive, neutral, and in-complete. each section will be given assesed and placed into the closest fitting section.


0. Data Prep:
  Docuemnts are parsed into sections and tokenized, will parse markdown documents by indication of a new section, so either multiple new lines, section seperation line, or an inidcation of a title of a section.
  Each section will be tokenized and stored in the train/test datasets respectivly. <br>
<br>



1. Scoring with Q+A model and BERT Classification Results:

<br>
  1.1 Govern - 25% : <br>
Does the local government have an AI governance policy? 0/1 pt.<br>
Is there a designated AI Risk Officer of Governance Committee?<br>
Are AI decision-making policies publicly accessible?<br>
Does the government track all AI use cases across departments?<br>
Are AI projects evaluated for legal and regulatory compliance before deployment?<br>
Is there a process for approving new AI initiatives in government services?<br>
Are elected officials trained on AI risks and governance?<br>
Are third-party AI vendors required to follow transparency guidelines?<br>
Does the government document and log all AI-driven decisions?<br>
Is there a public grievance process for challenging AI-driven government decisions?<br>
<br>

  1.2 Map - 20% : <br>
Is there a complete inventory of all AI models used in government services?<br>
Are AI models classified based on risk level (e.g., high-risk for policing, low-risk for chatbots)?<br>
Does the government assess AI risks before deployment?<br>
Are AI-driven financial decisions reviewed for fairness?<br>
Are AI models in sensitive areas (policing, welfare) regularly audited?<br>
Does the government perform privacy impact assessments on AI models?<br>
Are AI risks communicated to the public and key stakeholders?<br>
Does the government require explainability for AI-driven public service decisions?<br>
Does AI use comply with federal and state AI governance laws?<br>
Is there a documented AI risk assessment process before the launch of new AI projects?<br>
<br>

  1.3 Measure - 30% : <br>
Are AI models tested for fairness, accuracy, and security before deployment?<br>
Are explainability techniques used to make AI decisions interpretable?<br>
Are financial AI models tested for taxation, fines, and resource allocation bias?<br>
Does the government use continuous monitoring systems to track AI model performance?<br>
Are AI models regularly audited for drift or unintended consequences?<br>
Are AI security risks (adversarial attacks, data poisoning) assessed before deployment?<br>
Does the government conduct independent external AI audits?<br>
Is there public reporting on AI model performance and impact?<br>
Are there standardized benchmarks for AI fairness and effectiveness in local government?<br>
Are AI security risks assessed before deployment?<br>
<br>

  1.4 Manage - 25% : <br>
Is there a risk mitigation strategy in place for AI failures?<br>
Are there processes for handling incorrect or harmful AI-driven decisions?<br>
Are AI models retrained regularly to reflect new data and evolving risks?<br>
Does the government seek public feedback on AI decision-making?<br>
Is there an AI incident response plan?<br>
Are AI models monitored for unexpected drift or bias post-deployment?<br>
Does the government require third-party AI vendors to follow risk management protocols?<br>
Are AI regulations in place for law enforcement or criminal justice applications?<br>
Does the government publish AI transparency reports?<br>
Are there documented case studies of AI-related mistakes and their resolutions?<br>
<br>

  1.5 Final Score=(∑(Total Questions in CategoryCategory Score​×Category Weight))×100 <br>

Govern Score: (Σ Govern Responses ÷ Total Govern Questions) × 0.25<br>
Map Score: (Σ Map Responses ÷ Total Map Questions) × 0.20<br>
Measure Score: (Σ Measure Responses ÷ Total Measure Questions) × 0.30<br>
Manage Score: (Σ Manage Responses ÷ Total Manage Questions) × 0.25<br>








## 0. Setup

### 0.1. Libraries and Helper Functions



In [2]:
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Now we are ready to do the imports.

In [3]:
#@title Imports

import numpy as np

import transformers
import evaluate

from datasets import load_dataset
from datasets import Dataset
from torchinfo import summary

from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import re
import pandas as pd

### 0.2 Data Preperation


We will use the policies in the "AI_Policy_deepseek" folder since they are all already in markdown format.

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

#deepseek
file_path = "drive/MyDrive/datasci266/finalProject/AI_Policy_deepseek/"
myStates = ['NY', 'CA', 'SD', 'AZ', 'WA', 'FL', 'TX']
fileName = "_AI_Policy_deepseek.md"

#claude
file_path2 = "drive/MyDrive/datasci266/finalProject/dataGov_claude/"
myStates2 = ['WY', 'UT', 'CO', 'NJ', 'ME', 'AR', 'MT']
fileName2 = "_dataGov_Policy_claude.md"


Mounted at /content/drive


In [ ]:

# num is the pattern to match, 0 -> deepseek, 1 -> claude
def fileParser(cur_file, num):
  patterns = [r'\n---\n', r'#{2,3}']
  match_locs = []
  cont = []

  with open(cur_file, 'r') as file:
    content = file.read()
    p = patterns[num]
    matches = list(re.finditer(p, content))
    match_locs.append([match.end() for match in matches])
    for i in range(len(match_locs[0])-1):
      #print(content[match_locs[0][i]:match_locs[0][i+1]])
      cont.append(content[match_locs[0][i]:match_locs[0][i+1]])


  print(f"matches found: \n{match_locs}")
  return match_locs, cont

x, y = fileParser(file_path2 + myStates2[0] + fileName2, 1)



matches found: 
[[55, 77, 482, 1291, 1334, 1666, 1883, 2062, 2103, 2298, 2702, 2745, 3024, 3235, 3268, 3511, 3896, 4185, 4500, 4548, 4758, 4992, 5261, 5295, 5577, 5826, 6049, 6085, 6362, 6635, 6680, 7091, 7274, 7494, 7555, 7764, 7984, 8184, 8372]]


Now that we have all the docs and the locations where to split them, now we need to look at them and give some labels tp create the training/test sets. <br>
we'll take a look at the content.

In [ ]:
#cur_file = file_path + myStates[0] + fileName

def myCall(file_path, myStates, fileName, num):
  parsed_loc = {}
  parsed_content = []
  df = pd.DataFrame()


  for state in myStates:
    cur_file = file_path + state + fileName
    print(cur_file)
    parsed_loc[state], parsed_content = fileParser(cur_file, num)
    new_rec = pd.DataFrame({"text": parsed_content})
    df = pd.concat([df, new_rec], ignore_index=True)

  print(df['text'].shape)
  print(df['text'])
  return df

deepseek_df = myCall(file_path, myStates, fileName, 0)
claude_df = myCall(file_path2, myStates2, fileName2, 1)
df = pd.concat([deepseek_df, claude_df])


In [ ]:
# once we get all the data out, last step is to add the empty labels/

# 0: active, 1: proactive, 2: neutral, 3: incomplete
labels = ['active', 'proactive', 'neutral', 'incomplete']
tempVals = [[0,0,0,0]]*df['text'].shape[0]
combo = pd.DataFrame(tempVals, columns=labels)
df[labels] = combo
df.tail()

#df.to_csv('output.csv', mode='a', header=True, index=True)

,text,active,proactive,neutral,incomplete
290,12. Training and Awareness\n\n**12.1 Required...,0,0,0,0
291,13. Compliance and Audit\n\n**13.1 Monitoring...,0,0,0,0
292,14. Policy Exceptions\n\n**14.1 Exception Pro...,0,0,0,0
293,15. Policy Review and Updates\n\nThis policy ...,0,0,0,0
294,16. Enforcement\n\nViolations of this policy ...,0,0,0,0



Now that we have created the data frame, I have gone and labels the dataset to make it complete with labels for training/testing.
<br>
made one hot for each of the data points.


In [ ]:
labels = ['Active', 'Proactive', 'Neutral', 'Incomplete']
fp = "drive/MyDrive/datasci266/resp2.csv"
resp_df = pd.read_csv(fp)
# print(resp_df.head())

for i in range(resp_df.shape[0]):
  # clean strings
  dirty = str(resp_df.iloc[i]['text'])
  clean = re.sub(r'[^a-zA-Z0-0\s]', '', dirty)
  resp_df.at[i, 'text'] = clean

  # update 1 hots
  if labels[0] in resp_df.iloc[i]["response"]:
    resp_df.at[i, 'active'] = 1
  elif labels[1] in resp_df.iloc[i]["response"]:
    resp_df.at[i, 'proactive'] = 1
  elif labels[2] in resp_df.iloc[i]["response"]:
    resp_df.at[i, 'neutral'] = 1
  else:
    resp_df.at[i, 'incomplete'] = 1

final_df = pd.DataFrame()
final_df['text'] = resp_df['text']
final_df['active'] = resp_df['active']
final_df['proactive'] = resp_df['proactive']
final_df['neutral'] = resp_df['neutral']
final_df['incomplete'] = resp_df['incomplete']


print(f"active: {final_df['active'].value_counts()}")
print(f"proactive: {final_df['proactive'].value_counts()}")
print(f"neutral: {final_df['neutral'].value_counts()}")
print(f"incomplete: {final_df['incomplete'].value_counts()}")

final_df.to_csv('clean_final_df.csv')



active: active
0    210
1    159
Name: count, dtype: int64
proactive: proactive
0    369
Name: count, dtype: int64
neutral: neutral
1    197
0    172
Name: count, dtype: int64
incomplete: incomplete
0    356
1     13
Name: count, dtype: int64



Now, we can start to tokenize/encode the data to prep to feed into BERT model.
we will stick with the GPT4All tokenizer since that is what we use for the summarization/ Q+A model.



In [ ]:

final_df = pd.read_csv('clean_final_df.csv')
final_df = final_df.drop(columns=['Unnamed: 0'])
new_df = pd.DataFrame()
for i in range(final_df.shape[0]):
  new_df.at[i, 'text'] = final_df.iloc[i]['text']
  if final_df.iloc[i]['active'] == 1 or final_df.iloc[i]['proactive'] == 1:
    new_df.at[i, 'labels'] = 1
  elif final_df.iloc[i]['neutral'] == 1:
    new_df.at[i, 'labels'] = 2
  else:
    new_df.at[i, 'labels'] = 3

final_df = new_df
print(final_df.head())
print(final_df.shape)

# train test split


# train_df = final_df.sample(frac=0.8, random_state=42)
# test_df = final_df.drop(train_df.index)

# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

# train_dataset[:16]

                                                text  labels
0  \n  Purpose  \nThis policy establishes guideli...     2.0
1  \n  Scope  \nThis policy applies to all employ...     2.0
2  \n  Legal and Regulatory Compliance  \nAll use...     2.0
3  \n  Ethical Principles  \nThe use of AI tools ...     2.0
4  \n  Permitted Uses  \nAI tools may be used for...     2.0
(369, 2)



**Final Steps of Processing ** <br>
this is the best it is going to get, because I am not going to spend time crafting a dataset with the correct classes as I do not have enough time for this. <br>

I have not found much documentation on how to properly craft these datasets and its looking to be a lot more work than i initially thought. So, I will just document what the labels map to without creating the 'ClassLabel' in my dataset to properly store this information.<br>

Here, we have the cleaned text with its assigned label: <br>
{'active':1, 'neutral':2, 'incomplete':3}
<br><br>
Then, i create the train and test split sets.


In [ ]:


test = {}
temp = list(final_df['text'])
final_df['labels'] = final_df['labels'].astype(int)
temp2 = list(final_df['labels'])
label_names = {'active':1, 'neutral':2, 'incomplete':3}
test['text'] = temp
test['labels'] = temp2

print(final_df.groupby('labels').count())

# #print(test)


        text
labels      
1        159
2        197
3         13


in order for this dataset to be trainable, i need more incomplete:3 examples, because right now it is not possible to batch these samples to train a model on because there is not enough representation to sufficent training/batching. <br><br>
this is a problem that i might be able to solve if i process the new examples Lawrence uploaded, but there no telling if i'll have enough incomplete examples.

In [ ]:

full_dataset = Dataset.from_dict(test)

# full_dataset[:4]
# print(type(final_df.at[0, 'labels']))
# create train and test sets
split_dataset = full_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

split_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 295
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 74
    })
})

**Trying a new dataset** <br><br>

Since there are some issues with this custom one, I will try a different dataset from hugging face called [legalbench](https://huggingface.co/datasets/nguha/legalbench).<br>
This pre-trained [bert-base model](https://huggingface.co/mauro/bert-base-uncased-finetuned-clause-type) might be useful later. <br>

lets take a look at the dataset.<br><br>

As cool as this is, I found another cool dataset to help with our goal of understandning policy and contract documents. Lets take a look at [CUAD](https://github.com/TheAtticusProject/cuad), Contract Understanding Atticus Dataset. <br>
CUAD has some pre-trained models like RoBERTa-base, which we have worked with RoBERT before. Lets see about the CUAD set and pre-trained models-<br>
DO NOT LOAD THE CUAD SET!! crashed my session and my notebook is still fucked up a little bit from it lol. <br><br>

Instead, we will look at the [AGORA](https://zenodo.org/records/15053471), AI GOvernance and Regulatory Archive dataset.

In [ ]:
# legalbench_dataset = load_dataset("nguha/legalbench",'contract_nli_confidentiality_of_agreement')
# legalbench_dataset

# DO NOT RUN, CRASHED SESSION
# #CUAD_load = load_dataset("theatticusproject/cuad")
# #CUAD_load

####################### break #######################

# AGORA datasets, /documents.csv/ is like an index to navigate the /fulltext/ dir.
agora_root_path = 'drive/MyDrive/datasci266/agora/'
agora_docs_path = agora_root_path+'documents.csv'
agora_fulltext_path = agora_root_path+'fulltext/'

agora_documents_main = pd.read_csv(agora_docs_path)
agora_documents_main.head()
# agora_documents.columns

,AGORA ID,Official name,Casual name,Link to document,Authority,Collections,Most recent activity,Most recent activity date,Proposed date,Annotated?,...,"Strategies: Licensing, registration, and certification",Strategies: New institution,Strategies: Performance requirements,Strategies: Pilots and testbeds,Strategies: Tiering,Strategies: Tiering: Tiering based on domain of application,Strategies: Tiering: Tiering based on generality,Strategies: Tiering: Tiering based on impact,Strategies: Tiering: Tiering based on inputs,Strategies: Tiering: Tiering based on planning ability
0,1087,Artificial Intelligence and the Information Pr...,Artificial Intelligence and the Information Pr...,https://privacy.org.nz/assets/New-order/Resour...,Government of New Zealand,Miscellaneous documents,Proposed,2023-09-21,2023-09-21,False,...,False,False,False,False,False,False,False,False,False,False
1,1748,Servicemember Quality of Life Improvement and ...,"FY2025 NDAA, Section 1032 (""Competitive demons...",https://www.congress.gov/bill/118th-congress/h...,United States Congress,FY2025 NDAA;NDAA provisions,Enacted,2024-12-23,2024-12-11,True,...,False,False,False,False,False,False,False,False,False,False
2,1109,Artificial Intelligence (Regulation) Act 2024,Artificial Intelligence (Regulation) Act 2024,https://bills.parliament.uk/publications/53068...,Government of the United Kingdom,Miscellaneous documents,Proposed,2024-05-10,2023-11-22,True,...,False,True,False,True,False,False,False,False,False,False
3,1803,Freedom Online Coalition Joint Statement on Re...,Freedom Online Coalition Joint Statement on Re...,https://pl.usembassy.gov/freedom-online-coalit...,Other multinational,Multinational,Enacted,2024-09-23,2024-09-23,True,...,False,False,True,False,False,False,False,False,False,False
4,2040,Google DeepMind Frontier Safety Framework Vers...,Google DeepMind Frontier Safety Framework Vers...,https://storage.googleapis.com/deepmind-media/...,Private-sector companies,Corporate policies and commitments;Editors' Picks,Enacted,2025-02-04,2025-02-04,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# now lets extract the relevent files
# we want docs that belong to the following collections:
collection_names = ['U.S. regulations, executive orders, and agency policies', 'U.S. state and local documents', 'U.S. federal laws', 'Corporate policies and commitments']
# if we need more docs, add 'Corporate policies and commitments' to collection_names
# before corp policies, shape: 611, 100; after shape: 622,100

# col names: 'AGORA ID', 'Collections', 'Short summary', 'Tags', 'Official name',
# consider, 'Tags' have a high prob of being null!
agora_documents = pd.DataFrame()
agora_documents = agora_documents_main[agora_documents_main['Collections'].isin(collection_names)]
# agora_documents.shape
agora_docs_filtered = agora_documents[['AGORA ID', 'Official name', 'Short summary', 'Collections', 'Tags']]
agora_docs_filtered.head()

# this is the end of using the /documents.csv/.


,AGORA ID,Official name,Short summary,Collections,Tags
5,456,Candidate Voice Fraud Prohibition Act,Amends the Federal Election Campaign Act to pr...,U.S. federal laws,Harms: Detrimental content;Harms: Violation of...
6,2002,New York City Department of Consumer and Worke...,Requires New York City employers to conduct an...,U.S. state and local documents,NaN
7,2019,Boise Regulations 4.30q City Use of Artificial...,Requires Boise's city departments and contract...,U.S. state and local documents,NaN
8,2031,City of Long Beach Data Privacy Guidelines (2021),"Requires Long Beach to ensure transparency, ac...",U.S. state and local documents,NaN
9,2025,"Indianapolis City Council Proposal No.362, 2023",Establishes a study commission to review and r...,U.S. state and local documents,NaN


In [ ]:
# now that we have the AGORA ID of the text we want, lets extract them and agregate them.

def fileContent(cur_file):
  try:
    with open(cur_file, 'r') as file:
      content = file.read()
  except:
    content = 'NONE'
    print(f"{cur_file} not found")
  return content


count = 0
for id in agora_docs_filtered['AGORA ID']:
  print(f"id: {id}")
  content = fileContent(agora_fulltext_path + str(id) + '.txt')
  # get current index in pd dataframe, update with text col, content
  index = agora_docs_filtered.index[agora_docs_filtered['AGORA ID'] == id].tolist()[0]
  agora_docs_filtered.at[index, 'text'] = content

agora_docs_filtered.head()



In [ ]:
# now, i'll make me a new CSV with just the data i want so we can make a huggingface compatible dataset.
# lets make sure this set is clean, some txt files were not found so i'll want to delete those out of my df
# before converting to a CSV to save. we'll start with 622 records, then take out the few that are 'NONE'.
# only 3 were 'NONE' so now we have 619 records.

# justInCase_none = agora_docs_filtered[agora_docs_filtered['text'] == 'NONE']
# justInCase_none.head()

#agora_docs_filtered.to_csv('agora_docs_filtered_checkpoint.csv')
justInCase = agora_docs_filtered[agora_docs_filtered['text'] != 'NONE']
justInCase.tail()

# my final cleaned csv to save for later use
# justInCase.to_csv('AGORA_data_cleaned.csv')

# lets take a look at how many records actually have tags
# only 197 out of the remaining 619 have tags. this might not be a feature we can use for any meaningful training.
# filter_tags = justInCase[justInCase['Tags'].notna()]
# filter_tags.shape


,AGORA ID,Official name,Short summary,Collections,Tags,text
801,1730,An Act Authorizing And Adjusting Bonds Of The ...,Establishes an AI education tool pilot program...,U.S. state and local documents,NaN,"Sec. 143. (Effective July 1, 2024) (a) For the..."
802,1416,Indiana House Bill 1047,"Defines ""computer generated image"" as those cr...",U.S. state and local documents,NaN,AN ACT to amend the Indiana Code concerning cr...
803,1537,Washington House Bill 1999,Defines digitization to include AI use in crea...,U.S. state and local documents,NaN,AN ACT Relating to fabricated intimate or sexu...
804,1505,Illinois Senate Bill 382,Amends the Act to include liability exceptions...,U.S. state and local documents,NaN,AN ACT concerning civil law.\n \nBe it enacted...
805,1734,"California AB 2355 (2024), Section 8",Requires political advertisements generated or...,U.S. state and local documents,NaN,SEC. 8. Section 84514 is added to the Governme...


**Craft Dataset with Huggingface** <br><br>

now we'll have to streamline loading the cleaned AGORA csv i created above into a split dataset for 'Train', 'Validate', and 'Test'. <br>
[datasets documentation](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/loading_methods)

In [4]:
# from datasets import Features

# clean_agora_data_path = 'drive/MyDrive/datasci266/agora/AGORA_data_cleaned.csv'
# agora_main_dataset = load_dataset('csv', data_files=clean_agora_data_path, split="train")
# agora_main_dataset = agora_main_dataset.remove_columns(['Unnamed: 0'])
# agora_split = agora_main_dataset.train_test_split(test_size=0.2)
# # agora_split

# agora_train_dataset = agora_split['train'].shuffle()
# agora_test_dataset = agora_split['test'].shuffle().select(range(30))

# agora_train_dataset

my_agora = pd.read_csv("AGORA_data_cleaned.csv")


my_agora.shape

(619, 7)

In [22]:

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = str(doc)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    return doc

my_agora['text'] = my_agora['text'].apply(normalize_document)
my_agora['Short summary'] = my_agora['Short summary'].apply(normalize_document)


sen1 = []
sen2 = []

for p in my_agora['text']:
  if type(p) != float:
    j = p.split('.')
    for k in j:
      if k != '' and len(k)>10:
        kk = re.sub(r"\n", "", k)
        sen1.append(kk)

for t in my_agora['Short summary']:
  if type(t) != float:
    k = t.split('.')
    for t in k:
      if t != '' and len(t)>10:
        tt = re.sub("\n", "", t)
        sen2.append(tt)

print(f"s1 length: {len(sen1)}")
print(f"s2 length: {len(sen2)}")

sen11 = []
sen22 = []

for i in range(len(sen1)):
  if len(sen1[i]) > 10:
    sen11.append(sen1[i])

for i in range(len(sen2)):
  if len(sen2[i]) > 10:
    sen22.append(sen2[i])

smol = min(len(sen11), len(sen22))
big = max(len(sen11), len(sen22))

leftover = (big-smol)//2
sent1 = sen11[:smol+leftover]
sent2 = sen22+sen11[smol+leftover:]


agora_sen_df = pd.DataFrame({'sentence1':sent1, 'sentence2':sent2})


agora_sen_df.shape

s1 length: 4760
s2 length: 618


(2670, 2)

In [27]:
agora_sen_df.to_csv('agora_sentences.csv')



**this looks to be okay** <br><br>

now lets try preparing this data to be learned upon. <br><br>

look at [kraken](https://huggingface.co/cognitivecomputations/Kraken) which is a combination of models for reasoning and carriing out sophisticated ML tasks. maybe we can have a few models who are 'experts' on these topics. <br><br>

this is a promising feature extraction model [data2vec](https://huggingface.co/facebook/data2vec-text-base). masks input to predict latent representations of the full data input by leverating a self-distiliation setup using transformer arch. [fine-tuned applications](https://huggingface.co/models?other=data2vec-text) of this data2vec.<br>

through feature extraction, we can process these specific text examples and build build representations that are specific to this corpus.

In [ ]:

# pipe = pipeline('feature-extraction', model='facebook/data2vec-text-base')

# d2v_tokenizer = AutoTokenizer.from_pretrained("facebook/data2vec-text-base")
# d2v_model = AutoModel.from_pretrained("facebook/data2vec-text-base")


config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecTextModel were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecTextModel were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

In [ ]:

# test_input = agora_train_dataset[0]['text']
# print(f"test_dataset: \n{test_input}")

# test_output = pipe(test_input, return_tensors=True)
# test_output.shape
#### can use this for the data2vec above ####




test_dataset: 
SECTION 5. AND BE IT FURTHER ENACTED, That the Interagency Commission
on School Construction, on or before December 15, 2024, shall report to the General
Assembly, in accordance with § 2–1257 of the State Government Article, on the eligibility for school construction funding for artificial intelligence weapon detection systems. 


torch.Size([1, 70, 768])

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = AutoModel.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# text from dataset
t_input = [test_dataset[0]['text'], test_dataset[1]['text']]
print(f"test_dataset: \n{test_dataset[0]['text']}")
# test tokenizer
token_input = bert_tokenizer(t_input, max_length=100, truncation=True, padding='max_length', return_tensors="pt")
token_input



test_dataset: 
  Data Quality Standards
 Establish data quality metrics for accuracy completeness and timeliness
 Implement validation rules for data entry
 Conduct regular data quality assessments
 Document procedures for addressing data quality issues




{'input_ids': tensor([[  101,  7154, 14801, 14365,   142,  8419,  1830, 10550,  2233,  3068,
         12676,  1116,  1111, 10893,  2335,  1757,  1105, 21169,  3954,   146,
         26318,  1880,  9221,  1891,  2995,  1111,  2233,  3990, 16752, 13890,
          2366,  2233,  3068, 25451, 11387, 15447,  2227,  8826,  1111, 13735,
          2233,  3068,  2492,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 16752,  2050,  2875, 23404,  1204,  5399, 16752,  2050,  2875,
         23404,  2145,  1138,  1103,  1268,  1106, 11737,  1147,  2357,  2233,
         11336, 16437, 22590,  1104, 

In [ ]:
# test model input
b_output = bert_model(**token_input)
b_output


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4160,  0.0040,  0.0276,  ..., -0.2175,  0.2809,  0.1556],
         [ 0.5896,  0.2400,  0.4683,  ...,  0.6919,  0.1296,  0.2132],
         [-0.4949, -0.5053,  0.0643,  ...,  0.1755,  0.1393,  0.2438],
         ...,
         [ 0.1061, -0.1866,  0.1832,  ..., -0.0709,  0.3288,  0.2269],
         [-0.0407, -0.1753,  0.1666,  ..., -0.0889,  0.4001,  0.2331],
         [-0.0446, -0.3224,  0.1955,  ..., -0.1468,  0.2717,  0.2708]],

        [[ 0.4268, -0.4356, -0.0319,  ...,  0.0032,  0.0339, -0.1109],
         [ 0.5243, -0.6763,  0.3193,  ...,  0.3799,  0.1848, -0.4101],
         [-0.1919,  0.0489, -0.9129,  ...,  0.6467, -1.3954, -0.4989],
         ...,
         [-0.3820, -0.3460, -0.0792,  ..., -0.0331,  0.0477,  0.0245],
         [-0.3401, -0.4687, -0.0630,  ..., -0.1229,  0.0645,  0.1398],
         [-0.1701,  0.1370,  0.1867,  ..., -0.0130,  0.0500,  0.2346]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [ ]:

# time to tokenize
# here, i am processing the data to get the input ids and attention masks for each data point.
# from there, i can craft pytorch to split into batchs. this wasn't working before b/c i didn't
# take this step in extracting the ids and att. masks.
MAX_SEQUENCE_LENGTH = 128

def tokenize_text(data):

  encoded = bert_tokenizer.batch_encode_plus(
          data,
          add_special_tokens=True,
          max_length=MAX_SEQUENCE_LENGTH,
          padding='max_length',
          truncation=True,
          return_attention_mask=True,
          return_token_type_ids=True,
          return_tensors="pt"
      )
  input_ids = encoded['input_ids']
  attention_masks = encoded['attention_mask']

  return input_ids, attention_masks




AttributeError: 'Dataset' object has no attribute 'groupby'

In [ ]:
import torch




train_input_ids, train_att_masks = tokenize_text(train_dataset['text']).values.tolist()
test_input_ids, test_att_masks = tokenize_text(test_dataset['text']).values.tolist()

# convert labels into tensors
train_y = torch.LongTensor(train_dataset['labels'].values.tolist())
test_y = torch.LongTensor(test_dataset['labels'].values.tolist())

# lets have a look
train_y.size()


AttributeError: 'tuple' object has no attribute 'values'